# Ontology interface

This tutorial demonstrates basic usages of __owlready2__ API for ontology manipulation. Notably, new ontology concept triple classes (subject, predicate, object) will be dynamically created, with optional existing ontology parent classes that are loaded from an OWL ontology. Then through the interconnected relations specified in triples, designators and their corresponding ontology concepts can be double-way queried for input information in certain tasks, eg. making a robot motion plan. 

In [1]:
from pathlib import Path
from typing import Type, TYPE_CHECKING
import pycram
from pycram.designator import ObjectDesignatorDescription

Unknown attribute "type" in /robot[@name='pr2']/link[@name='base_laser_link']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='wide_stereo_optical_frame']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='narrow_stereo_optical_frame']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='laser_tilt_link']
Failed to import Giskard messages
Could not import RoboKudo messages, RoboKudo interface could not be initialized
pybullet build time: Nov 28 2023 23:51:11


# Owlready2

[Owlready2](https://owlready2.readthedocs.io/en/latest/intro.html) is a Python package providing a transparent access to OWL ontologies. It supports various manipulation operations, including but not limited to loading, modification, saving ontologies. Built-in supported reasoners include [HermiT](http://www.hermit-reasoner.com) and [Pellet](https://github.com/stardog-union/pellet).

In [2]:
import logging
try:
    import owlready2
    from owlready2 import *
except ImportError:
    owlready2 = None
    logging.error("Could not import owlready2, Ontology Manager could not be initialized!")

logging.getLogger().setLevel(logging.INFO)

# Ontology Manager

`OntologyManager` is the singleton class acting as the main interface between PyCram with ontologies, whereby object instances in the former could query relevant information based on the semantic connection with their corresponding ontology concepts.

Such connection, as represented by triples (subject-predicate-object), could be also created on the fly if not pre-existing in the loaded ontology.

Also new and updated concepts with their properties defined in runtime could be stored into an [SQLite3 file database](https://owlready2.readthedocs.io/en/latest/world.html) for reuse.

Here we will use [SOMA ontology](https://ease-crc.github.io/soma) as the baseline to utilize the generalized concepts provided by it.

In [3]:
from pycram.ontology.ontology import OntologyManager, SOMA_HOME_ONTOLOGY_IRI
from pycram.ontology.ontology_common import OntologyConceptHolderStore, OntologyConceptHolder

ontology_manager = OntologyManager(SOMA_HOME_ONTOLOGY_IRI)
main_ontology = ontology_manager.main_ontology
soma = ontology_manager.soma
dul = ontology_manager.dul

[INFO] [1712825298.616830]: Ontology [http://www.ease-crc.org/ont/SOMA-HOME.owl#]'s name: SOMA-HOME has been loaded
[INFO] [1712825298.617596]: - main namespace: SOMA-HOME
[INFO] [1712825298.618148]: - loaded ontologies:
[INFO] [1712825298.618654]: http://www.ease-crc.org/ont/SOMA-HOME.owl#
[INFO] [1712825298.619127]: http://www.ease-crc.org/ont/DUL.owl#
[INFO] [1712825298.619599]: http://www.ease-crc.org/ont/SOMA.owl#


## Ontology Concept Holder
__`OntologyConceptHolder`__ class, encapsulating an __`owlready2.Thing`__ instance, is used primarily as the binding connection between the `owlready2.Thing` ontology concept to PyCram designators. We make it that way, instead of creating a custom concept class that inherits from `owlready2.Thing` for the reasons below:

- `owlready2` API does not have very robust support for client classes to inherit from theirs with added (non-semantic) attributes, particularly in our case, where classes like `DesignatorDescription` have their `metaclass` as `ABCMeta`, while it is `EntityClass` that is the metaclass used for basically all concepts (classes, properties) in `owlready2`. Since those two metaclasses just bear no relationship, for the inheritance to work, the only way is to create a child metaclass with both of those as parents, however without full support by `owlready2`, plus the second reason below will point out it's not worth the effort.


- Essentially, we will have new ontology concept classes created dynamically, if their types inherit from `owlready2.Thing`, all custom non-semantic (of types known only by PyCram) attributes, which are defined by their own in child classes, will apparently be not savable into the ontology by `owlready2` api. Then the next time the ontology is loaded, those same dynamic classes will not be created anymore, thus without those attributes either, causing running error.

As such, in short, an ontology concept class, either newly created on the fly or loaded from ontologies, has to be `owlready2.Thing` or its pure derived class (without non-semantic attributes), so to make itself reusable upon reloading.

Notable attributes:
- `ontology_concept`: An ontology concept of `owlready2.Thing` type or its pure child class (without custom non-semantic attributes), either dynamically created, or loaded from an ontology
- `designators`: a list of `DesignatorDescription` instances associated with `ontology_concept`
- `resolve`: a `Callable` typically returning a list of `DesignatorDescription` as specific designators, like `designators` or its subset, inferred from the ontology concept. In fact, it can be resolved to anything else relevant, up to the caller.

## Query ontology classes and their properties

Classes in the loaded ontology can be queried based on their exact names, or part of them, or by namespace.
Here, we can see essential info (ancestors, super/sub-classes, properties, direct instances, etc.) of the found ontology class.

In [4]:
ontology_designed_container_class = ontology_manager.get_ontology_class('DesignedContainer')
ontology_manager.print_ontology_class(ontology_designed_container_class)
classes = ontology_manager.get_ontology_classes_by_subname('PhysicalObject'); print(classes[0])
classes = ontology_manager.get_ontology_classes_by_namespace('SOMA'); print(classes[:2])

[INFO] [1712825298.740637]: -------------------
[INFO] [1712825298.741301]: SOMA.DesignedContainer <class 'owlready2.entity.ThingClass'>
[INFO] [1712825298.741699]: Super classes: [DUL.DesignedArtifact, DUL.DesignedArtifact, SOMA.hasDisposition.some(SOMA.Containment), SOMA.hasDisposition.some(SOMA.Containment)]
[INFO] [1712825298.742080]: Ancestors: {owl.Thing, DUL.Object, SOMA.DesignedContainer, DUL.DesignedArtifact, DUL.PhysicalObject, DUL.PhysicalArtifact, DUL.Entity}
[INFO] [1712825298.742406]: Subclasses: [SOMA.Bottle, SOMA.Crockery, SOMA.Box, SOMA.Building, SOMA.Carafe, SOMA.Cupboard, SOMA.Dishwasher, SOMA.Dispenser, SOMA.Drawer, SOMA.Jar, SOMA.Pack, SOMA.Oven, SOMA.Shaker, SOMA.Refrigerator, SOMA.TrashContainer, SOMA-HOME.CustomContainerConcept, SOMA-HOME.AnotherCustomContainerConcept, SOMA-HOME.OntologyPlaceHolderObject, SOMA-HOME.OntologyLiquidHolderObject]
[INFO] [1712825298.742819]: Properties: [rdf-schema.label, SOMA.hasDisposition, rdf-schema.isDefinedBy, rdf-schema.commen

__Descendants__ of an ontology class can be also queried by

In [5]:
ontology_manager.get_ontology_descendant_classes(ontology_designed_container_class)[:5]

[SOMA.Bottle, SOMA.DesignedContainer, SOMA.Bowl, SOMA.Crockery, SOMA.Box]

## Create a new ontology concept class and its individual

A new ontology class can be created dynamically as inheriting from an existing class in the loaded ontology.
Here we create the class and its instance, also known as [__individual__](https://owlready2.readthedocs.io/en/latest/class.html#creating-equivalent-classes) in ontology terms, which is then wrapped inside an `OntologyConceptHolder`.

In [6]:
ontology_custom_container_class = ontology_manager.create_ontology_concept_class('CustomContainerConcept',
                                                                                 ontology_designed_container_class)
custom_container_concept_holder = OntologyConceptHolder(ontology_custom_container_class(name='ontology_custom_container_concept',
                                                                                        namespace=main_ontology))

## Access ontology concept classes and individuals
All ontology classes created on the fly purely inherit (without added non-semantic attributes) from __`owlready2.Thing`__, and so share the same namespace with the loaded ontology instance, `main_ontology`. They can then be accessible through that namespace by __`main_ontology.<class_name>`__.
The same applies for individuals of those classes, accessible by __`main_ontology.<class_individual_name>`__

In [7]:
ontology_manager.print_ontology_class(main_ontology.CustomContainerConcept)
print(f"custom_container_concept is {main_ontology.ontology_custom_container_concept}: {custom_container_concept_holder.ontology_concept is main_ontology.ontology_custom_container_concept}")

[INFO] [1712825298.890220]: -------------------
[INFO] [1712825298.891301]: SOMA-HOME.CustomContainerConcept <class 'owlready2.entity.ThingClass'>
[INFO] [1712825298.892014]: Super classes: [SOMA.DesignedContainer, owl.Thing]
[INFO] [1712825298.892582]: Ancestors: {owl.Thing, SOMA-HOME.CustomContainerConcept, DUL.Object, SOMA.DesignedContainer, DUL.DesignedArtifact, DUL.PhysicalObject, DUL.PhysicalArtifact, DUL.Entity}
[INFO] [1712825298.893037]: Subclasses: []
[INFO] [1712825298.893832]: Properties: []
[INFO] [1712825298.894993]: Instances: [SOMA-HOME.ontology_custom_container_concept]
[INFO] [1712825298.895463]: Direct Instances: [SOMA-HOME.ontology_custom_container_concept]
[INFO] [1712825298.895874]: Inverse Restrictions: []
custom_container_concept is SOMA-HOME.ontology_custom_container_concept: True


For ones already existing in the ontology, they can only be accessed through their corresponding ontology, eg: `soma` as follows

In [8]:
ontology_manager.print_ontology_class(soma.Cup)

[INFO] [1712825298.916737]: -------------------
[INFO] [1712825298.917397]: SOMA.Cup <class 'owlready2.entity.ThingClass'>
[INFO] [1712825298.917950]: Super classes: [SOMA.Crockery, SOMA.hasPhysicalComponent.some(SOMA.DesignedHandle)]
[INFO] [1712825298.918460]: Ancestors: {owl.Thing, SOMA.Crockery, DUL.Object, SOMA.DesignedContainer, SOMA.Tableware, DUL.DesignedArtifact, SOMA.DesignedTool, DUL.PhysicalObject, SOMA.Cup, DUL.PhysicalArtifact, DUL.Entity}
[INFO] [1712825298.919121]: Subclasses: []
[INFO] [1712825298.919813]: Properties: [SOMA.hasPhysicalComponent, rdf-schema.isDefinedBy, rdf-schema.comment]
[INFO] [1712825298.920730]: Instances: []
[INFO] [1712825298.921243]: Direct Instances: []
[INFO] [1712825298.921731]: Inverse Restrictions: []


## Connect ontology class individuals with designators
After creating `custom_container_concept_holder` (wrapping `custom_container_concept` as an `owlready2.Thing`), we connect it to a designator (say `obj_designator`) by:
- Append to `obj_designator.ontology_concept_holders` with `custom_container_concept_holder`
- Append to `custom_container_concept_holder.designators` with `obj_designator`

In [9]:
custom_container_designator = ObjectDesignatorDescription(names=["obj"])
custom_container_designator.ontology_concept_holders.append(custom_container_concept_holder)
custom_container_concept_holder.designators.append(custom_container_designator)

We can also automatize all the above setup with a single function call

In [10]:
another_custom_container_designator = ontology_manager.create_ontology_linked_designator(designator_name="another_custom_container",
                                                                                         designator_class=ObjectDesignatorDescription,
                                                                                         ontology_concept_name="AnotherCustomContainerConcept",
                                                                                         ontology_parent_class=ontology_designed_container_class)
another_custom_container_concept = another_custom_container_designator.ontology_concept_holders[0].ontology_concept 
print(f"Ontology concept: {another_custom_container_concept.name} of class {type(another_custom_container_concept)}")
another_custom_container_designator = OntologyConceptHolderStore().get_ontology_concept_holder_by_name(main_ontology.AnotherCustomContainerConcept.instances()[0].name).get_default_designator()
print(f"Designator: {another_custom_container_designator.names[0]} of type {type(another_custom_container_designator)}")

Ontology concept: another_custom_container_concept of class SOMA-HOME.AnotherCustomContainerConcept
Designator: another_custom_container of type <class 'pycram.designator.ObjectDesignatorDescription'>


## Create new ontology triple classes

Concept classes of a triple, aka [__subject, predicate, object__], can be created dynamically. Here we will make an example creating ones for [__handheld objects__] and [__placeholder objects__], with a pair of predicate and inverse predicate signifying their mutual relation.

In [11]:
PLACEABLE_ON_PREDICATE_NAME = "placeable_on"
HOLD_OBJ_PREDICATE_NAME = "hold_obj"
ontology_manager.create_ontology_triple_classes(ontology_subject_parent_class=soma.DesignedContainer,
                                                subject_class_name="OntologyPlaceHolderObject",
                                                ontology_object_parent_class=soma.Shape,
                                                object_class_name="OntologyHandheldObject",
                                                predicate_name=PLACEABLE_ON_PREDICATE_NAME,
                                                inverse_predicate_name=HOLD_OBJ_PREDICATE_NAME,
                                                ontology_property_parent_class=soma.affordsBearer,
                                                ontology_inverse_property_parent_class=soma.isBearerAffordedBy)

There, we use `soma.DesignedContainer` & `soma.Shape`, existing concept in SOMA ontology, as the parent classes for the subject & object concepts respectively.
There is also a note that those classes will have the same namespace with `main_ontology`, so later on to be accessible through it.

Then now we define some instances of the newly created triple classes, and link them to object designators, again using __`ontology_manager.create_ontology_linked_designator()`__

In [12]:
def create_ontology_handheld_object_designator(object_name: str, ontology_parent_class: Type[owlready2.Thing]):
    return ontology_manager.create_ontology_linked_designator(designator_name=object_name,
                                                              designator_class=ObjectDesignatorDescription,
                                                              ontology_concept_name=f"Onto{object_name}",
                                                              ontology_parent_class=ontology_parent_class)
# Holdable Objects
cookie_box = create_ontology_handheld_object_designator("cookie_box", main_ontology.OntologyHandheldObject)
egg = create_ontology_handheld_object_designator("egg", main_ontology.OntologyHandheldObject)
    
# Placeholder objects
placeholders = [create_ontology_handheld_object_designator(object_name, main_ontology.OntologyPlaceHolderObject)
                for object_name in ['table', 'stool', 'shelf']]

egg_tray = create_ontology_handheld_object_designator("egg_tray", main_ontology.OntologyPlaceHolderObject)

### Create ontology relations

Now we will create ontology relations or predicates between __placeholder objects__ and __handheld objects__ with __`ontology_manager.set_ontology_relation()`__

In [13]:
for place_holder in placeholders:
    ontology_manager.set_ontology_relation(subject_designator=cookie_box, object_designator=place_holder,
                                           predicate_name=PLACEABLE_ON_PREDICATE_NAME)

ontology_manager.set_ontology_relation(subject_designator=egg_tray, object_designator=egg,
                                       predicate_name=HOLD_OBJ_PREDICATE_NAME)

True

## Query designators based on their ontology-concept relations

Now we can make queries for designators from designators, based on the relation among their corresponding ontology concepts setup above

In [14]:
print(f"{cookie_box.names}'s placeholder candidates:",
      f"""{[placeholder.names for placeholder in
            ontology_manager.get_designators_by_subject_predicate(subject=cookie_box,
                                                                  predicate_name=PLACEABLE_ON_PREDICATE_NAME)]}""")

print(f"{egg.names}'s placeholder candidates:",
      f"""{[placeholder.names for placeholder in
            ontology_manager.get_designators_by_subject_predicate(subject=egg,
                                                                  predicate_name=PLACEABLE_ON_PREDICATE_NAME)]}""")

for place_holder in placeholders:
    print(f"{place_holder.names} can hold:",
          f"""{[placeholder.names for placeholder in
                ontology_manager.get_designators_by_subject_predicate(subject=place_holder,
                                                                      predicate_name=HOLD_OBJ_PREDICATE_NAME)]}""")

print(f"{egg_tray.names} can hold:",
      f"""{[placeholder.names for placeholder in
            ontology_manager.get_designators_by_subject_predicate(subject=egg_tray,
                                                                  predicate_name=HOLD_OBJ_PREDICATE_NAME)]}""")

['cookie_box']'s placeholder candidates: [['table'], ['stool'], ['shelf']]
['egg']'s placeholder candidates: [['egg_tray']]
['table'] can hold: [['cookie_box']]
['stool'] can hold: [['cookie_box']]
['shelf'] can hold: [['cookie_box']]
['egg_tray'] can hold: [['egg']]


# Practical examples

## Example 1
How about creating ontology concept classes encapsulating `pycram.enums.ObjectType`? We can do it by:

In [15]:
from pycram.datastructures.enums import ObjectType

# Create a generic ontology concept class for edible objects
generic_edible_class = ontology_manager.create_ontology_concept_class('GenericEdible')

# Create a list of object designators sharing the same concept class as [generic_edible_class]
edible_obj_types = [ObjectType.MILK, ObjectType.BREAKFAST_CEREAL]
for object_type in ObjectType:
    if object_type in edible_obj_types:
        # Create a designator for the edible object
        ontology_manager.create_ontology_object_designator_from_type(object_type, generic_edible_class)

print(f'{generic_edible_class.name} object types:')
for edible_ontology_concept in generic_edible_class.direct_instances():
    print(edible_ontology_concept,
          [des.types for des in OntologyConceptHolderStore().get_ontology_concept_holder_by_name(edible_ontology_concept.name).designators])


GenericEdible object types:
SOMA-HOME.milk_concept [['milk']]
SOMA-HOME.breakfast_cereal_concept [['breakfast_cereal']]


## Example 2
We could also make use of relations between ontology concepts that designators are associated with, to enable more abstract inputs in robot motion plan.

In a similar style to the scenario of __placeholder objects__ and __handheld objects__ above, but with a little bit difference, we will ask the robot to query which content holders (eg. cup, pitcher, bowl) whereby a milk box could be pourable into.

Basically, we will provide an ontology-based implementation for the query:
 
`abstract_ontology_concept -> specific_objects_in_world?`

To achieve it, we will create triple classes and configure a customized `resolve()` for the abstract concept, which returns its associated specific designators.
These designators are then used to again resolve for the target objects of interest, which become the inputs to a robot motion plan.

### Setup simulated environment

In [16]:
from pycram.worlds.bullet_world import BulletWorld, Object
from pycram.datastructures.pose import Pose

from pycram.process_module import simulated_robot
from pycram.designators.action_designator import *
from pycram.designators.location_designator import *

world = BulletWorld()
plane = Object("floor", ObjectType.ENVIRONMENT, "plane.urdf")
kitchen = Object("kitchen", ObjectType.ENVIRONMENT, "kitchen.urdf")
pr2 = Object("pr2", ObjectType.ROBOT, "pr2.urdf")
kitchen_designator = ObjectDesignatorDescription(names=["kitchen"])
robot_designator = ObjectDesignatorDescription(names=["pr2"]).resolve()

Scalar element defined multiple times: limit
Scalar element defined multiple times: limit


### Create PourableObject-LiquidHolder triple ontology classes

In [17]:
POURABLE_INTO_PREDICATE_NAME = "pourable_into"
HOLD_LIQUID_PREDICATE_NAME = "hold_liquid"
ontology_manager.create_ontology_triple_classes(ontology_subject_parent_class=soma.DesignedContainer,
                                                subject_class_name="OntologyLiquidHolderObject",
                                                ontology_object_parent_class=soma.Shape,
                                                object_class_name="OntologyPourableObject",
                                                predicate_name=POURABLE_INTO_PREDICATE_NAME,
                                                inverse_predicate_name=HOLD_LIQUID_PREDICATE_NAME,
                                                ontology_property_parent_class=soma.affordsBearer,
                                                ontology_inverse_property_parent_class=soma.isBearerAffordedBy)

### Spawn a pourable object & liquid holders into the world and Create their designators

In [18]:
# Holdable obj
milk_box = Object("milk_box", ObjectType.MILK, "milk.stl")
milk_box_designator = create_ontology_handheld_object_designator(milk_box.name, main_ontology.OntologyPourableObject)

# Liquid-holders
cup = Object("cup", ObjectType.JEROEN_CUP, "jeroen_cup.stl", pose=Pose([1.4, 1, 0.9]))
bowl = Object("bowl", ObjectType.BOWL, "bowl.stl", pose=Pose([1.4, 0.5, 0.9]))
pitcher = Object("pitcher", ObjectType.GENERIC_OBJECT, "Static_MilkPitcher.stl", pose=Pose([1.4, 0, 0.9]))
milk_holders = [cup, bowl, pitcher]
milk_holder_designators = [create_ontology_handheld_object_designator(obj.name, main_ontology.OntologyLiquidHolderObject)
                           for obj in milk_holders]

### Create an ontology relation between the designators of the pourable object & its liquid holders

In [19]:
for milk_holder_desig in milk_holder_designators:
    ontology_manager.set_ontology_relation(subject_designator=milk_box_designator, object_designator=milk_holder_desig,
                                           predicate_name=POURABLE_INTO_PREDICATE_NAME)

### Set up `resolve` for the ontology concept of the pourable object

In [20]:
milk_box_concept_holder = milk_box_designator.ontology_concept_holders[0]
def milk_box_concept_resolve(): 
    object_designator = ontology_manager.get_designators_by_subject_predicate(subject=milk_box_designator, predicate_name=POURABLE_INTO_PREDICATE_NAME)[0]
    return object_designator, object_designator.resolve()

milk_box_concept_holder.resolve = milk_box_concept_resolve

Here, for demonstration purpose only, we specify the resolving result by __`milk_box_concept_holder`__ as __`cup`__, the first-registered (default) pourable-into target milk holder, utilizing the ontology relation setup above.

Now, we can query the milk box's target liquid holder by resolving `milk_box_concept_holder`

In [21]:
target_milk_holder_designator, target_milk_holder = milk_box_concept_holder.resolve()
print(f"Pickup target object: {target_milk_holder.name}, a place holder of {milk_box_designator.names}")

Pickup target object: cup, a place holder of ['milk_box']


### Robot picks up the target liquid holder

In [22]:
with simulated_robot:
    ParkArmsAction([Arms.BOTH]).resolve().perform()

    MoveTorsoAction([0.3]).resolve().perform()

    pickup_pose = CostmapLocation(target=target_milk_holder, reachable_for=robot_designator).resolve()
    pickup_arm = pickup_pose.reachable_arms[0]

    print(pickup_pose, pickup_arm)

    NavigateAction(target_locations=[pickup_pose.pose]).resolve().perform()

    PickUpAction(object_designator_description=target_milk_holder_designator, arms=[pickup_arm], grasps=["front"]).resolve().perform()

    ParkArmsAction([Arms.BOTH]).resolve().perform()

    place_island = SemanticCostmapLocation("kitchen_island_surface", kitchen_designator.resolve(), target_milk_holder_designator.resolve()).resolve()

    place_stand = CostmapLocation(place_island.pose, reachable_for=robot_designator, reachable_arm=pickup_arm).resolve()

    NavigateAction(target_locations=[place_stand.pose]).resolve().perform()

    PlaceAction(target_milk_holder_designator, target_locations=[place_island.pose], arms=[pickup_arm]).resolve().perform()

    ParkArmsAction([Arms.BOTH]).resolve().perform()
world.exit()

CostmapLocation.Location(pose=header: 
  seq: 0
  stamp: 
    secs: 1712825301
    nsecs: 538790225
  frame_id: "map"
pose: 
  position: 
    x: 0.6399999999999999
    y: 1.24
    z: 0.0
  orientation: 
    x: -0.0
    y: 0.0
    z: 0.15234391170286138
    w: -0.988327543159185, reachable_arms=['left', 'right']) left


# Save ontologies to an OWL file
After all the above operations to our ontologies, we now can save them to an OWL file on disk

In [23]:
ontology_manager.save(f"{Path.home()}/ontologies/New{main_ontology.name}.owl")

[INFO] [1712825308.787155]: Ontologies have been saved to /home/ducthan/ontologies/NewSOMA-HOME.owl


True

# Optimize ontology loading with SQLite3
Upon the initial ontology loading from OWL, an SQLite3 file is automatically created, acting as the quadstore cache for the loaded ontologies. This allows them to be __selectively__ reusable the next time that being loaded.
More info can be referenced [here](https://owlready2.readthedocs.io/en/latest/world.html). 